In [2]:
#Convert everything to JSON 
import json
import glob
import numpy as np
from joblib import dump, load
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
# from os import path, mkdir
import matplotlib.pyplot as plt


def convertoJson():
    with open('/Users/sloth/Desktop/CS271/271_project/DataMapping/keystroke_data_100.json', 'x') as F:
        # Use the json dumps method to write the list to disk
        all_data = {}
        for i in range(1,76):
            s = "00" + str(i) if i < 10 else "0" + str(i)
            all_files = glob.glob("/Users/sloth/Desktop/CS271/271_project/DataMapping/CNN/100/%s/*"%(s))
            data = []
            for file in all_files:
                data.append(list(np.load(file)))
            all_data["user"+str(i)] = data
        F.write(json.dumps(all_data))

def read_data(path):
    """ Reads the data file and returns as a dictionary """
    with open(path) as json_file:
        data = json.load(json_file)
    return data

def get_pos_neg(data, subject,count):
    """ Returns positive and negative data for a given subject"""
    x_pos = data[subject]
    x_neg = []
    for s in range(1,len(data)+1):
        # if s != count and x_neg_len > 0:
        #     x_neg.extend(data["user"+str(s)][:1])
        #     x_neg_len -= 2
        if s != count:
            x_neg.extend(data["user"+str(s)][:10])
    return x_pos.copy(), x_neg.copy()


In [3]:
# Extract data from each folder, and other samples from CNN/75/ folder, scale it, and use it to train the mdoels 

from os import path, mkdir
def partition_data_balanced(p):
    """ Partitions the data in class balanced form and saves in files 
    Utilizes all 400 of the positive samples, and randomly selects 
    400 negative samples to produce the subset"""
    data = read_data(p)
    scaler = StandardScaler()
    count = 1
    for s in data:
        x_pos, x_neg = get_pos_neg(data, s,count)
        x = []
        y = []
        x.extend(x_pos)
        y.extend([1 for _ in range(len(x_pos))] + [0 for _ in range(len(x_neg)-1)])
        for _ in range(len(x_neg)-1):
            r = np.random.randint(1, len(x_neg))
            x.append(x_neg.pop(r))
        # print(len(x))
        
        X = np.array(x, dtype = 'float64')
        Y = np.array(y,dtype = 'float64')
        x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)
        data_dir = "/Users/sloth/Desktop/CS271/271_project/DataMapping/balanced_new_data_" + str(p.split("_")[-1].split(".")[0]) + "/"
        if not path.isdir(data_dir):
            mkdir(data_dir)
        if not path.isdir(data_dir + "/" + s):
            mkdir(data_dir + "/" + s)
            
        scaler.fit(x_train)
        x_train_scale = scaler.transform(x_train)
        x_test_scale = scaler.transform(x_test)
        np.save(data_dir + "/" + s + "/x_train.npy", x_train_scale)
        np.save(data_dir + "/" + s + "/x_test.npy", x_test_scale)
        np.save(data_dir + "/" + s + "/y_train.npy", y_train)
        np.save(data_dir + "/" + s + "/y_test.npy", y_test)

        # Does PCA simultaneously
        # scaler.fit(X)
        # X = scaler.transform(X)
        pca = PCA(.95)
        pca.fit(x_train_scale)
        x_train_pca = pca.transform(x_train_scale)
        x_test_pca = pca.transform(x_test_scale)
        np.save(data_dir + "/" + s + "/x_pca_train.npy", x_train_pca)
        np.save(data_dir + "/" + s + "/x_pca_test.npy", x_test_pca)
        np.save(data_dir + "/" + s + "/y_pca_train.npy", y_train)
        np.save(data_dir + "/" + s + "/y_pca_test.npy", y_test)
        count +=1
    
#partition_data_balanced()

def get_training_data(subject,pca,keystrokes_size):
    """Returns the training data partition for the given subject"""

    path =  "/Users/sloth/Desktop/CS271/271_project/DataMapping/balanced_new_data_" + str(keystrokes_size) + "/"+ str(subject)
    if pca:
        x_path = path + "/x_pca_train.npy"
        y_path = path + "/y_pca_train.npy"
    else:
        x_path = path + "/x_train.npy"
        y_path = path + "/y_train.npy"

    x_train = np.load(x_path)
    y_train = np.load(y_path)

    return x_train.astype(float), y_train.astype(float)


def get_test_data(subject,pca, keystrokes_size):
    """Returns the training data partition for the given subject"""

    path =  "/Users/sloth/Desktop/CS271/271_project/DataMapping/balanced_new_data_" + str(keystrokes_size) + "/"+ str(subject)
    if pca:
        x_path = path + "/x_pca_test.npy"
        y_path = path + "/y_pca_test.npy"
    else:
        x_path = path + "/x_test.npy"
        y_path = path + "/y_test.npy"

    x_test = np.load(x_path)
    y_test = np.load(y_path)

    return x_test.astype(float), y_test.astype(float)

paths = ["/Users/sloth/Desktop/CS271/271_project/DataMapping/keystroke_new_data_50.json"]#"/Users/sloth/Desktop/CS271/271_project/DataMapping/keystroke_new_data_75.json","/Users/sloth/Desktop/CS271/271_project/DataMapping/keystroke_new_data_50.json"]
# paths = ["/Users/sloth/Downloads/UB_keystroke_dataset/DataMapping/keystroke_data_50.json","/Users/sloth/Downloads/UB_keystroke_dataset/DataMapping/keystroke_data_75.json","/Users/sloth/Downloads/UB_keystroke_dataset/DataMapping/keystroke_data_100.json"]
# for p in paths:
#     partition_data_balanced(p)

In [9]:

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, det_curve, roc_curve, confusion_matrix, precision_recall_fscore_support, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from logitboost import LogitBoost
# from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
# from IPython.display import Image
# from sklearn.tree import export_graphviz
from subprocess import check_call
from time import perf_counter



class Classical_Models:
    def __init__(self,path, balanced=False):
        self.path = path
        self.data = read_data(path)
        self.RF_averages = dict()
        self.KNN_averages = dict()
        self.SVM_averages = dict()
        self.LOG_averages = dict()


    def startTraining(self,pca, ratio):
        # Trains one type of model given flags
        self.svm_training_with_Bagging(pca,ratio)
        self.knn_training_with_Bagging(pca,ratio)
        self.log_training_with_LogitBoost(pca,ratio)
        #self.rf_training_with_Grid(pca, ratio)


    def rf_training_with_Grid(self, pca, ratio):
        print("~~~~~~~~~~Starting RF Gridsearch~~~~~~~~~~")

        time_data = []

        n_estimators = [int(x) for x in np.linspace(start=10, stop=10, num=10)]
        max_features = ["auto", "sqrt"]
        max_depth = list(range(5, 35, 10))

        min_samples_split = [2, 5, 10]
        min_samples_leaf = [1, 2, 4]
        bootstrap = [True,False]
        hyperparameters = dict(
            n_estimators=n_estimators,
            max_features=max_features,
            # max_depth=max_depth,
            # min_samples_split=min_samples_split,
            # min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
        )
        self.RF_averages["FAR"] = 0
        self.RF_averages["FNR"] = 0
        self.RF_averages["ACC"] = 0
        self.RF_averages["EER"] = 0
        count = 0
        for s in self.data:
            count += 1
            X_train, Y_train = get_training_data(s,pca,int(self.path.split("_")[-1].split(".")[0]))
            X_train = X_train.astype(float)
            Y_train = Y_train.astype(float)
            rf_clf = RandomForestClassifier()
            clf = GridSearchCV(rf_clf, hyperparameters, scoring="f1", n_jobs=-1)
            clf.fit(X_train, Y_train)
            # print("Done with RF_Grid for ", s)
            X_test, Y_test = get_test_data(s,pca,int(self.path.split("_")[-1].split(".")[0]))
            Y_pred = clf.predict(X_test)
            TP = 0
            FP = 0
            FN = 0
            TN = 0
            for num in range(len(Y_pred)):
                if(Y_pred[num] == 1.0):
                    if(Y_test[num] == 1.0):
                        TP += 1
                    else:
                        FP += 1
                else:
                    if(Y_test[num] == 1.0):
                        FN += 1
                    else:
                        TN += 1
            # print("TP",TP,"FP",FP,"FN",FN,"TN",TN)
            # print("Accuracy is", accuracy_score(Y_test,Y_pred))
            FAR = FP/(FP+TN)
            FNR = FN/(TP+FN)
            ACC = accuracy_score(Y_test,Y_pred)
             # print("PR-F: \n",precision_recall_fscore_support(Y_test,Y_pred, labels= np.unique(Y_pred)))
            fpr, tpr, thresholds = roc_curve(Y_test, Y_pred,pos_label=1.0)
            # Calculate the EER from the ROC curve
            eer_threshold = thresholds[np.nanargmin(np.absolute((fnr - fpr)))]
            fnr = 1 - tpr
            # EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
             # theoretically eer from fpr and eer from fnr should be identical but they can be slightly differ in reality
            eer_1 = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
            eer_2 = fnr[np.nanargmin(np.absolute((fnr - fpr)))]
            # return the mean of eer from fpr and from fnr
            EER = (eer_1 + eer_2) / 2
            self.RF_averages["FAR"] += FAR
            self.RF_averages["FNR"] += FNR
            self.RF_averages["ACC"] += ACC 
            self.RF_averages["EER"] += EER
        self.RF_averages["FAR"] /= count
        self.RF_averages["FNR"] /= count
        self.RF_averages["ACC"] /= count
        self.RF_averages["EER"] /= count
        print("EER: {:.2f}%".format((self.RF_averages["EER"]/count)* 100))  
        print("ACC: {:.2f}%".format((self.RF_averages["ACC"]/count)* 100))  
        print("FAR: {:.2f}%".format((self.RF_averages["FAR"]/count)* 100)) 
        print("FNR: {:.2f}%".format((self.RF_averages["FNR"]/count)* 100)) 
        # save_time_data(ratio, "RF", "RF_Grid", pca, "train", sum(time_data) / len(time_data))
        
        
    def svm_training_with_Bagging(self, pca, ratio):
        print("~~~~~~~~~~Starting SVM Bagging~~~~~~~~~~")
        count = 0
        best_model = None
        self.SVM_averages["FAR"] = 0
        self.SVM_averages["FNR"] = 0
        self.SVM_averages["ACC"] = 0
        self.SVM_averages["EER"] = 0
        
        for s in self.data:
            X_train, Y_train = get_training_data(s,pca,int(self.path.split("_")[-1].split(".")[0]))
            X_train = X_train.astype(float)
            Y_train = Y_train.astype(float)

            sv_clf = SVC(gamma="auto", decision_function_shape="ovr",tol=.001)
            bag_clf = BaggingClassifier(sv_clf, n_estimators=20)
            bag_clf.fit(X_train, Y_train)
            # data_dir = "/Users/sloth/Desktop/CS271/271_project/DataMapping/balanced_models/"
            # if not path.isdir(data_dir):
            #     mkdir(data_dir)
            # print("Done with SVM_Bagging for", s)
            X_test, Y_test = get_test_data(s,pca,int(self.path.split("_")[-1].split(".")[0]))
            Y_pred = bag_clf.predict(X_test)
            TP = 0
            FP = 0
            FN = 0
            TN = 0
            for num in range(len(Y_pred)):
                if(Y_pred[num] == 1.0):
                    if(Y_test[num] == 1.0):
                        TP += 1
                    else:
                        FP += 1
                else:
                    if(Y_test[num] == 1.0):
                        FN += 1
                    else:
                        TN += 1
            # print("TP",TP,"FP",FP,"FN",FN,"TN",TN)
            # print("Accuracy is", accuracy_score(Y_test,Y_pred))
            FAR = FP/(FP+TN)
            FNR = FN/(TP+FN)
            ACC = accuracy_score(Y_test,Y_pred)
            count += 1
            # print("PR-F: \n",precision_recall_fscore_support(Y_test,Y_pred, labels= np.unique(Y_pred)))
            fpr, tpr, thresholds = roc_curve(Y_test, Y_pred,pos_label=1.0)
            # Calculate the EER from the ROC curve
            fnr = 1 - tpr
            eer_threshold = thresholds[np.nanargmin(np.absolute((fnr - fpr)))]
             # theoretically eer from fpr and eer from fnr should be identical but they can be slightly differ in reality
            eer_1 = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
            eer_2 = fnr[np.nanargmin(np.absolute((fnr - fpr)))]
            # return the mean of eer from fpr and from fnr
            EER = (eer_1 + eer_2) / 2
            self.SVM_averages["FAR"] += FAR
            self.SVM_averages["FNR"] += FNR
            self.SVM_averages["ACC"] += ACC
            self.SVM_averages["EER"] += EER
        self.SVM_averages["FAR"] /= count
        self.SVM_averages["FNR"] /= count
        self.SVM_averages["ACC"] /= count
        self.SVM_averages["EER"] /= count
        print("EER: {:.2f}%".format((self.SVM_averages["EER"])* 100))  
        print("ACC: {:.2f}%".format((self.SVM_averages["ACC"])* 100))  
        print("FAR: {:.2f}%".format((self.SVM_averages["FAR"])* 100)) 
        print("FNR: {:.2f}%".format((self.SVM_averages["FNR"])* 100)) 
           

        


    def knn_training_with_Bagging(self, pca, ratio):
        print("~~~~~~~~~~Starting KNN Bagging~~~~~~~~~~")
        time_data = []
        self.KNN_averages["FAR"] = 0
        self.KNN_averages["FNR"] = 0
        self.KNN_averages["ACC"] = 0
        self.KNN_averages["EER"] = 0
        count = 0
        for s in self.data:
            X_train, Y_train = get_training_data(s, pca,int(self.path.split("_")[-1].split(".")[0]))
            X_train = X_train.astype(float)
            Y_train = Y_train.astype(float)
            bagging_clf = BaggingClassifier(
                KNeighborsClassifier(algorithm="brute", metric="minkowski")
            )
            bagging_clf.fit(X_train, Y_train)
            X_test, Y_test = get_test_data(s,pca,int(self.path.split("_")[-1].split(".")[0]))
            Y_pred = bagging_clf.predict(X_test)
            # threshold = 1.00 # set your desired threshold here
            # Y_scores = bagging_clf.predict_proba(X_test)[:,1] # get the predicted probabilities of positive class
            # Y_pred = (Y_scores > threshold).astype(int)
            # print("Done with KNN_Bagging for", s)
            # fpr, tpr, thresholds = roc_curve(Y_test, Y_pred)
            # print("FPR: ",fpr, "TPR: ",tpr,"Thresholds: ", thresholds)
            # print("Confusion Matrix\n",confusion_matrix(Y_test,Y_pred))
            # print("Accuracy is", accuracy_score(Y_test,Y_pred))
            # cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(Y_test,Y_pred), display_labels = [False, True])
            # cm_display.plot()
            # plt.show() 
            # print(X_test.shape) 
            TP = 0
            FP = 0
            FN = 0
            TN = 0
            for num in range(len(Y_pred)):
                if(Y_pred[num] == 1.0):
                    if(Y_test[num] == 1.0):
                        TP += 1
                    else:
                        FP += 1
                else:
                    if(Y_test[num] == 1.0):
                        FN += 1
                    else:
                        TN += 1
            FAR = FP/(FP+TN)
            FNR = FN/(TP+FN)
            
            ACC = accuracy_score(Y_test,Y_pred)
            count +=1
            # print("PR-F: \n",precision_recall_fscore_support(Y_test,Y_pred, labels= np.unique(Y_pred)))
            fpr, tpr, thresholds = roc_curve(Y_test, Y_pred)
            fnr = 1 - tpr
            eer_threshold = thresholds[np.nanargmin(np.absolute((fnr - fpr)))]
            # EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
             # theoretically eer from fpr and eer from fnr should be identical but they can be slightly differ in reality
            eer_1 = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
            eer_2 = fnr[np.nanargmin(np.absolute((fnr - fpr)))]
            # return the mean of eer from fpr and from fnr
            EER = (eer_1 + eer_2) / 2
            self.KNN_averages["EER"] += EER
            self.KNN_averages["FAR"] += FAR
            self.KNN_averages["FNR"] += FNR
            self.KNN_averages["ACC"] += ACC
        self.KNN_averages["FAR"] /= count
        self.KNN_averages["FNR"] /= count
        self.KNN_averages["ACC"] /= count
        self.KNN_averages["EER"] /= count
        print("EER: {:.2f}%".format((self.KNN_averages["EER"])* 100))  
        print("ACC: {:.2f}%".format((self.KNN_averages["ACC"])* 100))  
        print("FAR: {:.2f}%".format((self.KNN_averages["FAR"])* 100)) 
        print("FNR: {:.2f}%".format((self.KNN_averages["FNR"])* 100)) 
    
    def log_training_with_LogitBoost(self, pca, ratio):
        time_data = []

        print("~~~~~~~~~~Starting LogitBoost~~~~~~~~~~")
        count = 0
        self.LOG_averages["FAR"] = 0
        self.LOG_averages["FNR"] = 0
        self.LOG_averages["ACC"] = 0
        self.LOG_averages["EER"] = 0
        for s in self.data:
            X_train, Y_train = get_training_data(s, pca,int(self.path.split("_")[-1].split(".")[0]))
            X_train = X_train.astype(float)
            Y_train = Y_train.astype(float)

            lb_clf = LogisticRegression(penalty="l2")
            lb_clf.fit(X_train, Y_train)
            lb_clf.fit(X_train, Y_train)
            X_test, Y_test = get_test_data(s,pca,int(self.path.split("_")[-1].split(".")[0]))
            Y_pred = lb_clf.predict(X_test)    
            # print("Done with LOG_LBoost for", s)
            # fpr, tpr, thresholds = roc_curve(Y_test, Y_pred)
            # print("FPR: ",fpr, "TPR: ",tpr, "Thresholds: ", thresholds)
            TP = 0
            FP = 0
            FN = 0
            TN = 0
            for num in range(len(Y_pred)):
                if(Y_pred[num] == 1.0):
                    if(Y_test[num] == 1.0):
                        TP += 1
                    else:
                        FP += 1
                else:
                    if(Y_test[num] == 1.0):
                        FN += 1
                    else:
                        TN += 1
            FAR = FP/(FP+TN)
            FNR = FN/(TP+FN)
            ACC = accuracy_score(Y_test,Y_pred)
            count += 1
            # print("PR-F: \n",precision_recall_fscore_support(Y_test,Y_pred, labels= np.unique(Y_pred)))
            fpr, tpr, thresholds = roc_curve(Y_test, Y_pred,pos_label=1.0)
            # Calculate the EER from the ROC curve
            fnr = 1 - tpr
            eer_threshold = thresholds[np.nanargmin(np.absolute((fnr - fpr)))]
            # EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
             # theoretically eer from fpr and eer from fnr should be identical but they can be slightly differ in reality
            eer_1 = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
            eer_2 = fnr[np.nanargmin(np.absolute((fnr - fpr)))]
            # return the mean of eer from fpr and from fnr
            EER = (eer_1 + eer_2) / 2
            # print("EER: {:.2f}%".format(EER * 100))  
            self.LOG_averages["FAR"] += FAR
            self.LOG_averages["FNR"] += FNR
            self.LOG_averages["ACC"] += ACC
            self.LOG_averages["EER"] += EER
        self.LOG_averages["FAR"] /= count
        self.LOG_averages["FNR"] /= count
        self.LOG_averages["ACC"] /= count
        self.LOG_averages["EER"] /= count
        print("EER: {:.2f}%".format((self.LOG_averages["EER"])* 100))  
        print("ACC: {:.2f}%".format((self.LOG_averages["ACC"])* 100))  
        print("FAR: {:.2f}%".format((self.LOG_averages["FAR"])* 100)) 
        print("FNR: {:.2f}%".format((self.LOG_averages["FNR"])* 100)) 
            # print("Confusion Matrix\n",confusion_matrix(Y_test,Y_pred))
            # print("Accuracy is", accuracy_score(Y_test,Y_pred))
            # cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix(Y_test,Y_pred), display_labels = [False, True])
            # cm_display.plot()
            # plt.show()  
            
            
paths = ["/Users/sloth/Downloads/UB_keystroke_dataset/DataMapping/keystroke_new_data_100.json","/Users/sloth/Downloads/UB_keystroke_dataset/DataMapping/keystroke_new_data_75.json","/Users/sloth/Downloads/UB_keystroke_dataset/DataMapping/keystroke_new_data_50.json"]
#paths = ["/Users/sloth/Downloads/UB_keystroke_dataset/DataMapping/keystroke_data_50.json","/Users/sloth/Downloads/UB_keystroke_dataset/DataMapping/keystroke_data_75.json","/Users/sloth/Downloads/UB_keystroke_dataset/DataMapping/keystroke_data_100.json"]
for p in paths:
    rf = Classical_Models(p,balanced=True)
    rf.startTraining(False,50)
    # with open("/Users/sloth/Downloads/UB_keystroke_dataset/DataMapping/Results_data_pca_" + p[-7:-5],'x') as F:
    #     # Use the json dumps method to write the list to disk
    #     Results = {"SVM": rf.SVM_averages, "KNN": rf.KNN_averages, "LOG": rf.LOG_averages, "RF": rf.RF_averages}
    #     F.write(json.dumps(Results))
    

~~~~~~~~~~Starting SVM Bagging~~~~~~~~~~
EER: 49.92%
ACC: 93.10%
FAR: 0.00%
FNR: 99.84%
~~~~~~~~~~Starting KNN Bagging~~~~~~~~~~
EER: 47.88%
ACC: 92.59%
FAR: 0.96%
FNR: 94.80%
~~~~~~~~~~Starting LogitBoost~~~~~~~~~~
EER: 47.11%
ACC: 92.84%
FAR: 0.80%
FNR: 93.43%
~~~~~~~~~~Starting SVM Bagging~~~~~~~~~~
EER: 49.57%
ACC: 91.05%
FAR: 0.05%
FNR: 99.08%
~~~~~~~~~~Starting KNN Bagging~~~~~~~~~~
EER: 47.77%
ACC: 90.41%
FAR: 1.22%
FNR: 94.32%
~~~~~~~~~~Starting LogitBoost~~~~~~~~~~
EER: 44.75%
ACC: 91.04%
FAR: 1.10%
FNR: 88.40%
~~~~~~~~~~Starting SVM Bagging~~~~~~~~~~
EER: 42.66%
ACC: 88.46%
FAR: 0.56%
FNR: 84.76%
~~~~~~~~~~Starting KNN Bagging~~~~~~~~~~
EER: 24.05%
ACC: 90.88%
FAR: 3.79%
FNR: 44.30%
~~~~~~~~~~Starting LogitBoost~~~~~~~~~~
EER: 40.10%
ACC: 88.66%
FAR: 1.32%
FNR: 78.87%
